In [19]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [20]:
df = pd.read_csv("../data/preprocessed/final_etf_data/weekends_aggregated_embedding/XLE_v4.csv")
df = sign_next_day(df)
df = drop_sign_and_return(df)


In [21]:
df.head(10)

,Date,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.056630,...,-0.037106,0.018390,-0.048215,0.002478,-0.040920,-0.071706,0.038703,0.072361,1.0,1.0
1,2018-03-21,49.154427,0.072957,0.896156,0.030887,2.0,0.042071,-0.085665,0.049102,0.056413,...,-0.023607,-0.038444,-0.080668,-0.023549,0.033013,-0.076677,0.025544,0.023494,1.0,-1.0
2,2018-03-22,48.150974,0.088483,0.875179,0.036338,2.0,0.052145,0.009761,0.079683,-0.018089,...,-0.021881,0.021531,0.038033,0.002100,0.021596,-0.002760,-0.006045,-0.022443,1.0,-1.0
3,2018-03-23,47.857117,0.427724,0.518749,0.053528,2.0,0.374196,-0.083923,0.038409,0.053812,...,-0.066203,0.016252,-0.058235,0.001560,-0.006853,-0.029184,0.048074,0.090239,1.0,1.0
4,2018-03-26,48.717216,0.211071,0.502181,0.286748,3.0,-0.075677,-0.023936,0.040512,-0.017666,...,-0.049532,0.031493,-0.053943,-0.040137,-0.023122,-0.080216,-0.028291,0.058632,1.0,-1.0
5,2018-03-27,48.258488,0.103827,0.877021,0.019152,2.0,0.084675,-0.041524,-0.013985,0.035771,...,-0.021129,0.023080,-0.096873,-0.031458,-0.036633,-0.074150,-0.019721,0.054707,1.0,-1.0
6,2018-03-28,47.319557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,2018-03-29,48.315845,0.015619,0.060177,0.924203,2.0,-0.908584,0.043158,0.042428,0.058186,...,0.026012,0.011205,-0.075044,-0.072819,-0.024646,-0.081047,0.048397,0.021529,1.0,-1.0
8,2018-04-02,47.348217,0.552455,0.306189,0.141357,8.0,0.411098,-0.012684,-0.031329,0.033414,...,-0.019442,0.030787,-0.047978,-0.021097,-0.055515,-0.058128,0.014756,0.053678,1.0,1.0
9,2018-04-03,48.358829,0.866997,0.101776,0.031226,2.0,0.835771,-0.028622,-0.046466,0.002435,...,-0.035980,-0.044318,-0.011511,0.002219,0.006495,-0.081839,0.057890,0.046853,1.0,-1.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Columns: 393 entries, Date to Sign_next_day
dtypes: float64(392), object(1)
memory usage: 1.8+ MB


In [23]:
df.describe()

,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,emb_3,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
count,587.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,...,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.0,586.000000
mean,45.020384,0.268587,0.293812,0.437601,4.822695,-0.169014,-0.025843,-0.012444,0.049210,0.036937,...,-0.034748,0.015467,-0.045946,-0.033428,-0.010958,-0.063912,0.014257,0.035499,1.0,0.029010
std,8.338886,0.195841,0.185615,0.242011,4.342419,0.399241,0.026305,0.026705,0.027856,0.024961,...,0.024584,0.024443,0.024919,0.026553,0.030271,0.025759,0.028008,0.027793,0.0,0.996152
min,18.770016,0.009654,0.013320,0.009845,1.000000,-0.946141,-0.141979,-0.114503,-0.105180,-0.062333,...,-0.113508,-0.071967,-0.126134,-0.174583,-0.139759,-0.162657,-0.068274,-0.050797,1.0,-1.000000
25%,43.111204,0.107623,0.135345,0.277203,2.000000,-0.436588,-0.040575,-0.029236,0.033171,0.022061,...,-0.050836,-0.000347,-0.061723,-0.048235,-0.029691,-0.079411,-0.003211,0.019179,1.0,-1.000000
50%,46.764362,0.240177,0.295116,0.427987,4.000000,-0.165907,-0.025868,-0.011447,0.049679,0.037596,...,-0.035403,0.016457,-0.046241,-0.033696,-0.011419,-0.064627,0.014780,0.036926,1.0,1.000000
75%,49.901524,0.383273,0.412348,0.599388,6.000000,0.071896,-0.008555,0.003246,0.064123,0.052817,...,-0.019856,0.030888,-0.031723,-0.017178,0.007235,-0.046979,0.032090,0.053115,1.0,1.000000
max,56.558403,0.952804,0.923695,0.962437,50.000000,0.935071,0.068755,0.079683,0.142569,0.121486,...,0.087040,0.083685,0.042978,0.066864,0.105745,0.017902,0.115765,0.125414,1.0,1.000000


In [24]:
df.isna().sum()

Date                 0
Price                0
avg_positive_XLE    23
avg_neutral_XLE     23
avg_negative_XLE    23
                    ..
emb_381             23
emb_382             23
emb_383             23
is_trading_day      23
Sign_next_day        1
Length: 393, dtype: int64

In [25]:
df.nunique()

Date                587
Price               571
avg_positive_XLE    564
avg_neutral_XLE     564
avg_negative_XLE    564
                   ... 
emb_381             564
emb_382             564
emb_383             564
is_trading_day        1
Sign_next_day         3
Length: 393, dtype: int64

### defining target - I want to assign:

class 1 to sign_next_day ==1

and class 0 to sign_next_day ∈ {0, -1}

In [26]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,Date,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.056630,...,-0.037106,0.018390,-0.048215,0.002478,-0.040920,-0.071706,0.038703,0.072361,1.0,1
1,2018-03-21,49.154427,0.072957,0.896156,0.030887,2.0,0.042071,-0.085665,0.049102,0.056413,...,-0.023607,-0.038444,-0.080668,-0.023549,0.033013,-0.076677,0.025544,0.023494,1.0,0
2,2018-03-22,48.150974,0.088483,0.875179,0.036338,2.0,0.052145,0.009761,0.079683,-0.018089,...,-0.021881,0.021531,0.038033,0.002100,0.021596,-0.002760,-0.006045,-0.022443,1.0,0
3,2018-03-23,47.857117,0.427724,0.518749,0.053528,2.0,0.374196,-0.083923,0.038409,0.053812,...,-0.066203,0.016252,-0.058235,0.001560,-0.006853,-0.029184,0.048074,0.090239,1.0,1
4,2018-03-26,48.717216,0.211071,0.502181,0.286748,3.0,-0.075677,-0.023936,0.040512,-0.017666,...,-0.049532,0.031493,-0.053943,-0.040137,-0.023122,-0.080216,-0.028291,0.058632,1.0,0


In [27]:
df.shape

(586, 393)

In [28]:
df.isna().sum()

Date                 0
Price                0
avg_positive_XLE    23
avg_neutral_XLE     23
avg_negative_XLE    23
                    ..
emb_381             23
emb_382             23
emb_383             23
is_trading_day      23
Sign_next_day        0
Length: 393, dtype: int64

In [29]:
df.head()

,Date,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.056630,...,-0.037106,0.018390,-0.048215,0.002478,-0.040920,-0.071706,0.038703,0.072361,1.0,1
1,2018-03-21,49.154427,0.072957,0.896156,0.030887,2.0,0.042071,-0.085665,0.049102,0.056413,...,-0.023607,-0.038444,-0.080668,-0.023549,0.033013,-0.076677,0.025544,0.023494,1.0,0
2,2018-03-22,48.150974,0.088483,0.875179,0.036338,2.0,0.052145,0.009761,0.079683,-0.018089,...,-0.021881,0.021531,0.038033,0.002100,0.021596,-0.002760,-0.006045,-0.022443,1.0,0
3,2018-03-23,47.857117,0.427724,0.518749,0.053528,2.0,0.374196,-0.083923,0.038409,0.053812,...,-0.066203,0.016252,-0.058235,0.001560,-0.006853,-0.029184,0.048074,0.090239,1.0,1
4,2018-03-26,48.717216,0.211071,0.502181,0.286748,3.0,-0.075677,-0.023936,0.040512,-0.017666,...,-0.049532,0.031493,-0.053943,-0.040137,-0.023122,-0.080216,-0.028291,0.058632,1.0,0


In [30]:
df_catboost = df.copy()
df_catboost.head()

,Date,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day
0,2018-03-20,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.056630,...,-0.037106,0.018390,-0.048215,0.002478,-0.040920,-0.071706,0.038703,0.072361,1.0,1
1,2018-03-21,49.154427,0.072957,0.896156,0.030887,2.0,0.042071,-0.085665,0.049102,0.056413,...,-0.023607,-0.038444,-0.080668,-0.023549,0.033013,-0.076677,0.025544,0.023494,1.0,0
2,2018-03-22,48.150974,0.088483,0.875179,0.036338,2.0,0.052145,0.009761,0.079683,-0.018089,...,-0.021881,0.021531,0.038033,0.002100,0.021596,-0.002760,-0.006045,-0.022443,1.0,0
3,2018-03-23,47.857117,0.427724,0.518749,0.053528,2.0,0.374196,-0.083923,0.038409,0.053812,...,-0.066203,0.016252,-0.058235,0.001560,-0.006853,-0.029184,0.048074,0.090239,1.0,1
4,2018-03-26,48.717216,0.211071,0.502181,0.286748,3.0,-0.075677,-0.023936,0.040512,-0.017666,...,-0.049532,0.031493,-0.053943,-0.040137,-0.023122,-0.080216,-0.028291,0.058632,1.0,0


In [31]:
df_catboost["Date"] = pd.to_datetime(df_catboost["Date"]).dt.normalize()
df_catboost = df_catboost.sort_values("Date").reset_index(drop=True)

df_catboost["no_news"] = df_catboost["n_XLE"].isna().astype(int) # a day where there were no sector specific news
df_catboost.head(10)


,Date,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,2018-03-20,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.056630,...,0.018390,-0.048215,0.002478,-0.040920,-0.071706,0.038703,0.072361,1.0,1,0
1,2018-03-21,49.154427,0.072957,0.896156,0.030887,2.0,0.042071,-0.085665,0.049102,0.056413,...,-0.038444,-0.080668,-0.023549,0.033013,-0.076677,0.025544,0.023494,1.0,0,0
2,2018-03-22,48.150974,0.088483,0.875179,0.036338,2.0,0.052145,0.009761,0.079683,-0.018089,...,0.021531,0.038033,0.002100,0.021596,-0.002760,-0.006045,-0.022443,1.0,0,0
3,2018-03-23,47.857117,0.427724,0.518749,0.053528,2.0,0.374196,-0.083923,0.038409,0.053812,...,0.016252,-0.058235,0.001560,-0.006853,-0.029184,0.048074,0.090239,1.0,1,0
4,2018-03-26,48.717216,0.211071,0.502181,0.286748,3.0,-0.075677,-0.023936,0.040512,-0.017666,...,0.031493,-0.053943,-0.040137,-0.023122,-0.080216,-0.028291,0.058632,1.0,0,0
5,2018-03-27,48.258488,0.103827,0.877021,0.019152,2.0,0.084675,-0.041524,-0.013985,0.035771,...,0.023080,-0.096873,-0.031458,-0.036633,-0.074150,-0.019721,0.054707,1.0,0,0
6,2018-03-28,47.319557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
7,2018-03-29,48.315845,0.015619,0.060177,0.924203,2.0,-0.908584,0.043158,0.042428,0.058186,...,0.011205,-0.075044,-0.072819,-0.024646,-0.081047,0.048397,0.021529,1.0,0,0
8,2018-04-02,47.348217,0.552455,0.306189,0.141357,8.0,0.411098,-0.012684,-0.031329,0.033414,...,0.030787,-0.047978,-0.021097,-0.055515,-0.058128,0.014756,0.053678,1.0,1,0
9,2018-04-03,48.358829,0.866997,0.101776,0.031226,2.0,0.835771,-0.028622,-0.046466,0.002435,...,-0.044318,-0.011511,0.002219,0.006495,-0.081839,0.057890,0.046853,1.0,0,0


In [32]:
df_catboost = df_catboost.drop(columns=["Date"])
df_catboost.head(1)

,Price,avg_positive_XLE,avg_neutral_XLE,avg_negative_XLE,n_XLE,sent_index_XLE,emb_0,emb_1,emb_2,emb_3,...,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,is_trading_day,Sign_next_day,no_news
0,47.900124,0.617495,0.364768,0.017737,3.0,0.599758,-0.034911,-0.009287,0.05663,0.043724,...,0.01839,-0.048215,0.002478,-0.04092,-0.071706,0.038703,0.072361,1.0,1,0


In [ ]:
original_features = [
    'Price',
    'avg_positive_XLE',
    'avg_neutral_XLE', 
    'avg_negative_XLE',
    'n_XLE',
    'sent_index_XLE',
    'is_trading_day'
]


embedding_features = [f'emb_{i}' for i in range(384)]

target = 'Sign_next_day'
df_clean = df_catboost.dropna(subset=[target, 'is_trading_day']).copy()


for emb_col in embedding_features:
    df_clean[emb_col] = df_clean[emb_col].fillna(0)


df_clean['avg_positive_XLE'] = df_clean['avg_positive_XLE'].fillna(0)
df_clean['avg_neutral_XLE'] = df_clean['avg_neutral_XLE'].fillna(1)
df_clean['avg_negative_XLE'] = df_clean['avg_negative_XLE'].fillna(0)
df_clean['n_XLE'] = df_clean['n_XLE'].fillna(0)
df_clean['sent_index_XLE'] = df_clean['sent_index_XLE'].fillna(0)

print(f"Clean dataset size: {len(df_clean)} rows")



Clean dataset size: 563 rows


In [34]:
X = df_clean[original_features + embedding_features]
y = df_clean[target]

#y = (y == 1).astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 287, 0: 276}


In [35]:
embedding_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=12))  # 95% variance
])


original_pipeline = Pipeline([
    ('passthrough', FunctionTransformer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('original', original_pipeline, original_features),
        ('embeddings', embedding_pipeline, embedding_features)
    ],
    remainder='drop'
)

In [ ]:
catboost = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    train_dir="cb_tmp",
    allow_writing_files=False
)

full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', catboost)
])


param_distributions = {
    'preprocessor__embeddings__pca__n_components': [0.90, 0.95, 0.98, 50, 75, 100],
    'classifier__depth': randint(3, 7),
    'classifier__learning_rate': uniform(0.01, 0.19),
    'classifier__iterations': randint(50, 500),
    'classifier__l2_leaf_reg': uniform(1, 14),
    'classifier__border_count': randint(32, 128),
    'classifier__random_strength': uniform(0.5, 5),
    'classifier__min_data_in_leaf': randint(5, 30),
    'classifier__rsm': uniform(0.6, 0.4),
    'classifier__boosting_type': ['Ordered', 'Plain'],
    'classifier__bootstrap_type': ['Bernoulli', 'MVS'],
    'classifier__subsample': uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    n_iter=40,
    cv=5,
    scoring='roc_auc',
    random_state=12,
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)


random_search.fit(X, y)

print(f"Best AUC: {random_search.best_score_:.4f}")
print(f"\nBest parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")



Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC: 0.5781

Best parameters:
  classifier__boosting_type: Plain
  classifier__bootstrap_type: MVS
  classifier__border_count: 54
  classifier__depth: 6
  classifier__iterations: 132
  classifier__l2_leaf_reg: 2.5327958986412638
  classifier__learning_rate: 0.18682844273255841
  classifier__min_data_in_leaf: 24
  classifier__random_strength: 5.1117530126170045
  classifier__rsm: 0.6687112975911181
  classifier__subsample: 0.8036275793894408
  preprocessor__embeddings__pca__n_components: 0.95


In [37]:
best_preprocessor = random_search.best_estimator_.named_steps['preprocessor']
pca = best_preprocessor.named_transformers_['embeddings'].named_steps['pca']

print(f"Number of prncipals components kept: {pca.n_components_}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Original embedding dimensions (384) were reduced to {pca.n_components_}")


Number of prncipals components kept: 162
Explained variance ratio: 0.9507
Original embedding dimensions (384) were reduced to 162
